In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('train.csv', encoding='ISO-8859-1')
data.head()

,Id,Hotel_name,Review_Title,Review_Text,Rating
0,0,Park Hyatt,Refuge in Chennai,Excellent room and exercise facility. All arou...,80.0
1,1,Hilton Chennai,Hilton Chennai,Very comfortable and felt safe. \r\nStaff were...,100.0
2,2,The Royal Regency,No worth the rating shown in websites. Pricing...,Not worth the rating shown. Service is not goo...,71.0
3,3,Rivera,Good stay,"First of all nice & courteous staff, only one ...",86.0
4,4,Park Hyatt,Needs improvement,Overall ambience of the hotel is very good. In...,86.0


In [4]:
data.shape

(2351, 5)

In [7]:
data.isna().sum()

Id              0
Hotel_name      0
Review_Title    0
Review_Text     0
Rating          0
dtype: int64

In [8]:
data.dropna(inplace=True)

In [9]:
data.isna().sum()

Id              0
Hotel_name      0
Review_Title    0
Review_Text     0
Rating          0
dtype: int64

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
from nltk.corpus import stopwords
eng_stopwords = set(stopwords.words('english'))

In [13]:
print(data['Review_Text'].iloc[0])

Excellent room and exercise facility. All around atmosphere was calm and comfortable. Main dining room offers both excellent food and service. Avoid Flying Elephant restaurant, stick to the main dining room.


In [19]:
# !pip install pymorphy2

In [28]:
# !pip install catboost

In [27]:
import re
import pymorphy2
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

In [21]:
morph = pymorphy2.MorphAnalyzer()

In [23]:
def replace(text):
    text = text.replace('«', '')
    text = text.replace('»', '')
    text = text.replace('-', ' ')
    text = text.replace('-', ' ')
    text = text.replace(' 0 ', ' ')
    text = text.replace(' 1 ', ' ')
    text = text.replace(' 2 ', ' ')
    text = text.replace(' 3 ', ' ')
    text = text.replace(' 4 ', ' ')
    text = text.replace(' 5 ', ' ')
    text = text.replace(' 6 ', ' ')
    text = text.replace(' 7 ', ' ')
    text = text.replace(' 8 ', ' ')
    text = text.replace(' 9 ', ' ')
    text = text.replace('&', ' ')
    return text

def remove_urls(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if len(i) > 2:
            if i.strip().lower() not in eng_stopwords:
    #             final_text.append(i.strip())
                final_text.append(morph.parse(i)[0].normal_form.strip())
    return " ".join(final_text)

def lower_words(text):
    return str(text).lower()

def clean_text(text):
    text = lower_words(text)
    text = replace(text)
    text = remove_urls(text)
    text = remove_stopwords(text)
    return text

In [24]:
data['Clean_Review_Text'] = data['Review_Text'].apply(clean_text)

In [29]:
X = data['Clean_Review_Text']
Y = data['Rating']

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(X)
freq_term_matrix = count_vectorizer.transform(X)
tfidf = TfidfTransformer()
tf_idf_matrix = tfidf.fit_transform(freq_term_matrix)

In [33]:
count_list = tf_idf_matrix.toarray().sum(axis=0)
word_list = count_vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
word_freq = pd.DataFrame(count_list, index=word_list, columns=['Freq'])
word_freq.sort_values(by='Freq', ascending=False).head(10)

,Freq
good,167.850510
hotel,157.731491
room,88.994602
stay,86.405114
staff,73.383709
service,69.466089
food,68.730539
nice,62.208289
location,57.107304
rooms,54.574016


In [35]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_matrix, Y, shuffle=True, random_state=42)

In [36]:
model = CatBoostRegressor(verbose = False)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, pred))

MAE: 11.152246326345136
